### THIS SCRIPT WILL GENERATE WEATHER MODELS ###

* requires having the weather data first (downloaded daily)

In [1]:
import pickle
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold 
from sklearn.metrics import r2_score

In [3]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

In [4]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_pinball_loss, mean_squared_error

In [5]:
import libbloom as lb

In [6]:
from scipy.optimize import curve_fit

In [7]:
from scipy.stats import lognorm,expon

In [8]:
import glob
from datetime import date, timedelta

In [9]:
import warnings
warnings.filterwarnings("ignore")

In [10]:
## create several empty models with adabost

regr_1 = DecisionTreeRegressor(max_depth=4)

regr_2a = AdaBoostRegressor(
    DecisionTreeRegressor(max_depth=4), n_estimators=300, random_state=42
)
regr_2b = AdaBoostRegressor(
    DecisionTreeRegressor(max_depth=4), n_estimators=300, random_state=42
)
regr_2c = AdaBoostRegressor(
    DecisionTreeRegressor(max_depth=4), n_estimators=300, random_state=42
)


regr_3a = AdaBoostRegressor(
    DecisionTreeRegressor(max_depth=4), n_estimators=300, random_state=42
)
regr_3b = AdaBoostRegressor(
    DecisionTreeRegressor(max_depth=4), n_estimators=300, random_state=42
)
regr_3c = AdaBoostRegressor(
    DecisionTreeRegressor(max_depth=4), n_estimators=300, random_state=42
)

In [11]:
def specialTraining(X_train,y_train):
    ## created the training set and model for quantile regression
    all_models = {}
    common_params = dict(
        learning_rate=0.05,
        n_estimators=300,
        max_depth=3,
        min_samples_leaf=4,
        min_samples_split=4,
    )
    for alpha in [0.05, 0.5, 0.95]:
        gbr = GradientBoostingRegressor(loss="quantile", alpha=alpha, **common_params)
        all_models["q %1.2f" % alpha] = gbr.fit(X_train, y_train)
    return all_models

In [12]:
def logifunc(x,l,c,k):
    # logistic + constant
    global delta
    return l / (1 + c*np.exp(-k*x)) + delta 

In [13]:
def poli(x,a,b,c,d):
    # polynomial
    global delta
    return a*x**4 + b*x**3 + c*x**2 + d*x + delta 

In [14]:
def fitL(ds,polin =False):
    # fit polynomial or logistics to the data
    xdata = np.arange(len(ds))
    ydata = ds.values
    if(polin):
        popt, pcov = curve_fit(poli, xdata, ydata)
        a,b,c,d = popt
        ycs = []
        for x in xdata:
            ycs.append(poli(x,a,b,c,d) )
        dm = pd.DataFrame(index = ds.index, data = {'data' : ds.values, 'polinomial' : ycs})        
        
    else:
        popt, pcov = curve_fit(logifunc, xdata, ydata)
        l,c,k = popt
        ycs = []
        for x in xdata:
            ycs.append(logifunc(x,l,c,k) )
        dm = pd.DataFrame(index = ds.index, data = {'data' : ds.values, 'logistic' : ycs})
    return dm, popt

In [15]:
def getCelsius(F):
    #utility (should be moved in the lib)
    Celsius = (F - 32) * 5.0/9.0
    return Celsius

In [16]:
def createDD(dfWy,thresholds_pest):  
    ## creates degree days
    stations = dfWy.pnum.unique()
    dfAll = []
    for st in stations:
        df = dfWy[dfWy.pnum == st]
        dds = []
        for day,tmin,tmax,lat,lon,pres,elev,wspd in zip(df.date.values,df.tmin.values, 
                                                        df.tmax.values,df.lat.values, 
                                                        df.lon.values,
                                                        df.pres.values,
                                                        df.elev.values,
                                                        df.wspd.values):
            try:
                d = lb.computeDDaysSine(tmin,tmax, thresholds_pest['lower'], thresholds_pest['upper']) # call from the lib
            except:
                d = 0
            dds.append({'day': day, 'degree' : d, 'pnum' : st, 'lat' : lat, 'lon' : lon, 
                        'pres' : pres , 'elev':elev, 'wspd':wspd})
        ds = pd.DataFrame(dds)
        ds = ds.set_index('day')
        ds = ds.assign(ddays = ds.degree.cumsum()) ## ddays is for the insect not general ddays
        dfAll.append(ds)
    dfT = pd.concat(dfAll)
    dfT = dfT.fillna(0) ##
    return dfT

In [17]:
def getXY(dfT):
    # y = last point
    # get X, and Y for training
    stations = dfT.pnum.unique()
    ys = []
    xs = []
    for st in stations:
        df = dfT[dfT.pnum == st]
        df = df.fillna(method='ffill')
        dda = df.ddays.values[-1]
        x = df[['lat','lon','elev','pres','wspd']].values[-1]
        xs.append(x)
        ys.append(dda)
    return np.array(xs), np.array(ys).ravel()

In [18]:
def getXY_pres(dfT):
    # y = last point
    # simplified model to get the model for pressure 
    stations = dfT.pnum.unique()
    ys = []
    xs = []
    for st in stations:
        df = dfT[dfT.pnum == st]
        df = df.fillna(method='ffill')
        pr = df.pres.values[-1]
        x = df[['lat','lon','elev']].values[-1]
        xs.append(x)
        ys.append(pr)
    return np.array(xs), np.array(ys).ravel()

In [19]:
def getXY_wspd(dfT):
    # y = last point
    # simplified model to get the model for pressure 
    stations = dfT.pnum.unique()
    ys = []
    xs = []
    for st in stations:
        df = dfT[dfT.pnum == st]
        df = df.fillna(method='ffill')
        ws = df.wspd.values[-1]
        x = df[['lat','lon','elev']].values[-1]
        xs.append(x)
        ys.append(ws)
    return np.array(xs), np.array(ys).ravel()

In [20]:
def createModelsForDDPrediction(dfT,regr_2a,regr_2b,regr_2c,gbr = False):
    # model for winds from lat/lon/elev
    Xwind, ywind = getXY_wspd(dfT)
    model_wind = regr_2c.fit(Xwind, ywind)

    # model for pressure from lat/lon/elev
    Xpress, ypress = getXY_pres(dfT)
    model_press = regr_2b.fit(Xpress,ypress)
    
    # general model (incorporate wind/pressure/elevation)
    X, ys = getXY(dfT)
    if(gbr):
        #model_gen = regr_2a.fit(X,ys)
        all_models = specialTraining(X,ys)
        return all_models,model_press,model_wind
    else:
        model_gen = regr_2a.fit(X,ys)
        return model_gen,model_press,model_wind

In [21]:
def nestModels(point_and_elevation,model_gen,model_press,model_wind,gbr = False):
    # all models are nested to get the prediction
    # args:
    # point_and_elevation =  [lat, lon, elevation] as a numpy array shape (1,3)
    #
    # example 
    # pp = np.array([3.94824123e+01, -4.88099096e-01,  8.50000000e+01]).reshape(1,-1)
    #
    #
    def checkShape(s):
        if(s.shape == (1,3)):
            return True
        else:
            return False
    
    if(checkShape(point_and_elevation)):
        wind_prediction = model_wind.predict(point_and_elevation)[0]
        pressure_prediction = model_press.predict(point_and_elevation)[0]
        # nest the predictions 
        extended_point = np.append(point_and_elevation,[pressure_prediction,wind_prediction]).reshape(1,-1)
        
        if(gbr):
            ddps = []
            for md in model_gen.values():
                ddays_prediction = md.predict(extended_point)[0]
                ddps.append(ddays_prediction)
            return ddps
        else:
            ddays_prediction = model_gen.predict(extended_point)[0]
            return ddays_prediction
    else:
        print("point_and_elevation should have shape: (1,3)")
        return False

In [22]:
def saveModels(dfW,thresholds_spodoptera,outmodel, stop, start = '2023-01-01'): # the jst jan should be updated by 2024
    # save models
    models = {}
    for dm in pd.date_range(start=start,end=stop,freq='D'):
        dfWy = dfW[dfW.date <= dm]
        dfWy = dfWy.fillna(0) ## fillna with 0s
        regr_2a = AdaBoostRegressor(
            DecisionTreeRegressor(max_depth=4), n_estimators=300, random_state=42
        )
        regr_2b = AdaBoostRegressor(
            DecisionTreeRegressor(max_depth=4), n_estimators=300, random_state=42
        )
        regr_2c = AdaBoostRegressor(
            DecisionTreeRegressor(max_depth=4), n_estimators=300, random_state=42
        )
        try:
            dfT = createDD(dfWy,thresholds_spodoptera)
            model_gen,model_press,model_wind = createModelsForDDPrediction(dfT,regr_2a,regr_2b,regr_2c,True) ## uses ddays referring to yesterday
            models[dm] = {'model_gen' : model_gen, 'model_press' : model_press, 'model_wind' : model_wind}
            print(dm)
        except:
            pass
    pickle.dump(models,open(outmodel,"wb"))
    return models

In [23]:
thresholds_spodoptera = {'lower' : getCelsius(53.), 'upper' : getCelsius(95.)} ## others ! 

**Joins the weather files (since jan)**

In [7]:
wfiles = glob.glob("WeatherAndClimateData/WeatherSpain*.gz")

In [8]:
wfiles

['WeatherAndClimateData/WeatherSpain__2023-06-01 00:00:00__2023-07-01 14:59:21.415026.pickle.gz',
 'WeatherAndClimateData/WeatherSpain2023Jun01.pickle.gz']

In [26]:
dtmp =[]
for w in wfiles:
    dftmp = pd.read_pickle(w)
    dtmp.append(dftmp)
    

In [27]:
dfW = pd.concat(dtmp)
dfW = dfW.assign(date = dfW.index)
dfW = dfW.drop_duplicates()

In [28]:
%%time
dfT = createDD(dfW,thresholds_spodoptera) ## create dd data for all days till 2023

CPU times: user 1.17 s, sys: 0 ns, total: 1.17 s
Wall time: 1.17 s


In [31]:
today, yesterday,firstJan = lb.getUtilDates()


**This function must run once a day**

In [ ]:
savedir = "savedModels/spodoptera/" ## make it parametric for other pests
outmodel = savedir + "WeatherModels.pickle"

In [ ]:
models = saveModels(dfW,thresholds_spodoptera,outmodel, yesterday, start = firstJan)